# DQN implementation

This notebook implements a DQN - an approximate q-learning algorithm with experience replay and target networks. Trains the algorithm on openAI's gym, to breakout Atari game, and monitors its games by exporting videos.

For most problems, it is impractical to represent the $Q$-function as a table containing values for each combination of $s$ and $a$. Instead, we train a function approximator, such as a neural network with parameters $\theta$, to estimate the Q-values, i.e. $Q(s, a; \theta) \approx Q^*(s, a)$. This can done by minimizing the following loss at each step $i$:

$\begin{equation}L_i(\theta_i) = \mathbb{E}_{s, a, r, s'\sim \rho(.)} \left[ (y_i - Q(s, a; \theta_i))^2 \right]\end{equation}$ where $y_i = r +  \gamma \max_{a'} Q(s', a'; \theta_{i-1})$

Here, $y_i$ is called the TD (temporal difference) target, and $y_i - Q$ is called the TD error.

Note that the parameters from the previous iteration $\theta_{i-1}$ are fixed and not updated. In practice we use a snapshot of the network parameters from a few iterations ago instead of the last iteration. This copy is called the *target network*.

Q-Learning is an *off-policy* algorithm that learns about the greedy policy $a = \max_{a} Q(s, a; \theta)$ while using a different behaviour policy for acting in the environment/collecting data. This behaviour policy is usually an $\epsilon$-greedy policy that selects the greedy action with probability $1-\epsilon$ and a random action with probability $\epsilon$ to ensure good coverage of the state-action space.

### Experience Replay

To avoid computing the full expectation in the DQN loss, we can minimize it using stochastic gradient descent. If the loss is computed using just the last transition $\{s, a, r, s'\}$, this reduces to standard Q-Learning. 

The Atari DQN work introduced a technique called Experience Replay to make the network updates more stable. At each time step of data collection, the transitions are added to a circular buffer called the *replay buffer*. Then during training, instead of using just the latest transition to compute the loss and its gradient, we compute them using a mini-batch of transitions sampled from the replay buffer. This has two advantages: better data efficiency by reusing each transition in many updates, and better stability using uncorrelated transitions in a batch.

In [ ]:
import logging
from typing import Tuple

import gym
import numpy as np
import tensorflow as tf
from keras import Input
from keras.layers import Activation
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.models import Model, Sequential

Here is necessary constants, we will use them further during this file

In [ ]:
IM_SIZE = 84 # image size
N_EPISODES = 4000 # total number of episodes
N_STATE_FRAMES = 4 # number of used frames for developing state
BATCH_SIZE = 64 # batch size
MIN_BUFFER_SIZE = 16384 # minimal size of experience replay buffer, necessary count of experiences for starting training
MAX_BUFFER_SIZE = 65536 # maximal size of experience replay buffer
GAMMA = 0.99
TARGET_UPDATE_CYCLE = 10000 # number of steps, after which target model weights will be updated

In order to keep track of the data collected from the environment, we will use our own class ReplayBuffer. It stores experience data when we collect trajectories and is consumed during training.

This replay buffer is constructed using specs describing the tensors that are to be stored.

In [ ]:
class ReplayBuffer:
    def __init__(self,
                 max_size: int,
                 batch_size: int,
                 frame_width: int,
                 frame_height: int,
                 n_state_frames: int):
        self.max_size = max_size
        self.states = np.empty((self.max_size, frame_width, frame_height, n_state_frames), dtype=np.float32)
        self.actions = np.empty(self.max_size, dtype=np.uint8)
        self.rewards = np.empty(self.max_size, dtype=np.float32)
        self.flags = np.empty(self.max_size, dtype=np.bool)
        self.next_states = np.empty((self.max_size, frame_width, frame_height, n_state_frames), dtype=np.float32)
        self.batch_size = batch_size
        self.current_position = 0
        self.current_experience = 0

    def add_experience(self,
                       state: np.ndarray,
                       action: int,
                       reward: float,
                       done: bool,
                       next_state: np.ndarray):
        """
        Add experience which was received from environment.
        
        Parameters
        ----------
        state : np.ndarray with saved state. Current state.
        action : int received from gym environment. Chosen action (randomly or from model).
        reward : float, received from gym environment. Reward from current state.
        done: bool, flag of terminal state, received from gym environment.
        next_state: np.ndarray with saved state. Next state after performed action.
        """
        self.states[self.current_position, ...] = state
        self.actions[self.current_position] = action
        self.rewards[self.current_position] = reward
        self.flags[self.current_position] = done
        self.next_states[self.current_position, ...] = next_state
        self.current_position = (self.current_position + 1) % self.max_size
        if self.current_experience < self.max_size:
            self.current_experience += 1

    def get_mini_batch(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        
        if self.current_experience < self.batch_size:
            raise ValueError(f"""Not enough experience yet, required size = {self.batch_size}""")

        indices = np.random.choice(np.arange(self.current_experience), self.batch_size, replace=False)

        return (
            self.states[indices],
            self.actions[indices],
            self.rewards[indices],
            self.next_states[indices],
            self.flags[indices]
        )

## Deep Q-Network model

### Evaluation networks

We implemented light architecture of convolutional neural network. This model will be responsible for choosing particular actions, which depends on current state of the environment.


### Target networks

We also employ the so called "target network" - a copy of neural network weights to be used for reference Q-values:

The network itself is an exact copy of agent network, but it's parameters are not trained. Instead, they are moved here from agent's actual network every so often.

$$ Q_{reference}(s,a) = r + \gamma \cdot \max _{a'} Q_{target}(s',a') $$


Compute Q-learning TD error:

$$ L = { 1 \over N} \sum_i [ Q_{\theta}(s,a) - Q_{reference}(s,a) ] ^2 $$

With Q-reference defined as

$$ Q_{reference}(s,a) = r(s,a) + \gamma \cdot max_{a'} Q_{target}(s', a') $$

Where
* $Q_{target}(s',a')$ denotes q-value of next state and next action predicted by __target_network__
* $s, a, r, s'$ are current state, action, reward and next state respectively
* $\gamma$ is a discount factor.



In [ ]:
class DQN(Model):
    def __init__(self,
                 n_actions: int,
                 img_size: int,
                 gamma: float,
                 target_update_cycle: int,
                 n_state_frames: int):
        super(DQN, self).__init__()
        self.n_actions = n_actions
        self.img_size = img_size
        self.evaluation_network = self.build(self.img_size)
        self.target_network = self.build(self.img_size)
        self.optimizer = tf.keras.optimizers.Adam(0.01)
        self.gamma = gamma
        self.loss_function = tf.keras.losses.Huber()
        self.replay_buffer = ReplayBuffer(max_size=MAX_BUFFER_SIZE,
                                          batch_size=BATCH_SIZE,
                                          n_state_frames=n_state_frames,
                                          frame_width=self.img_size,
                                          frame_height=self.img_size)
        self.target_update_counter = 0
        self.target_update_cycle = target_update_cycle

    def build(self, img_size):
        dqn_model = Sequential(
            [
                Conv2D(32, 8, 8, kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2)),
                Activation('relu'),
                Conv2D(64, 4, 4, kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2)),
                Activation('relu'),
                Conv2D(128, 2, 2, kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2)),
                Activation('relu'),
                Flatten(),
                Dense(512, kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2)),
                Activation('relu'),
                Dense(self.n_actions, kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2))
            ]
        )
        return dqn_model

    def train(self):
        states, rewards, actions, next_states, flags = self.replay_buffer.get_mini_batch()
        with tf.GradientTape() as tape:
            q_eval_arr = self.evaluation_network(states)
            q_eval = tf.reduce_max(q_eval_arr, axis=1)
            target_q_values = self.target_network(next_states)
            discount_factor = tf.reduce_max(target_q_values)
            q_target = rewards + self.gamma * discount_factor
            loss = tf.reduce_mean(self.loss_function(q_eval, q_target))

        gradients = tape.gradient(loss, self.evaluation_network.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.evaluation_network.trainable_variables))
        self.target_update_counter += 1

        if self.target_update_counter % self.target_update_cycle == 0:
            self.target_network.set_weights(self.evaluation_network.get_weights())

    def add_experience(self, state, action, reward, done, next_state):
        self.replay_buffer.add_experience(state, action, reward, done, next_state)

### Processing game image 

Raw atari images are large, 210x160x3 by default. However, we don't need that level of detail in order to learn them.

We can thus save a lot of time by preprocessing game image, including
* Resizing to a smaller shape, 80 x 80
* Converting to grayscale
* Cropping irrelevant image parts (top & bottom)

In [ ]:
class ImageTransformer:
    def __init__(self, im_size, method):
        self.im_size = im_size
        self.method = method

    def transform(self, image) -> tf.Tensor:
        output = tf.image.rgb_to_grayscale(image)
        output = tf.image.crop_to_bounding_box(output, 34, 0, 160, 160)
        output = tf.image.resize(output,
                                 [self.im_size, self.im_size],
                                 method=self.method)
        output = tf.squeeze(output)
        output = output.numpy() / 255.0
        return output

Simple function for updating state

In [ ]:
def update_state(state, obs_small):
    return np.append(state[:, :, 1:], np.expand_dims(obs_small, 2), axis=2)

### Data Collection
Now execute the random policy in the environment for a few steps, recording the data in the replay buffer.

In [ ]:
def gain_minimal_experience(model: DQN,
                            img_transformer: ImageTransformer):
    env = gym.make('Breakout-v0')
    current_experience = 0
    obs = env.reset()
    small_obs = img_transformer.transform(obs)
    state = np.stack([small_obs] * 4, axis=2)
    done = False
    while current_experience < MIN_BUFFER_SIZE:
        action = env.action_space.sample()
        current_experience += 1
        obs, reward, done, _ = env.step(action)
        obs_small = img_transformer.transform(obs)
        next_state = update_state(state, obs_small)
        model.add_experience(state, action, reward, done, next_state)
        if done:
            obs = env.reset()
            small_obs = img_transformer.transform(obs)
            state = np.stack([small_obs] * 4, axis=2)
    env.close()
            
    return model

And we are ready to learn & play. We will learn our model through episodes with our experience replay buffer, make random sample from buffer every time, and learn model from this batch of data.

In [ ]:
if __name__ == '__main__':
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    env = gym.make('Breakout-v0')
    N_ACTIONS = env.action_space.n
    INITIAL_EPSILON = 0.4
    FINAL_EPSILON = 0.01
    EPSILON_DECAY = 1000000
    TRAINING_CYCLE = 2000
    epsilon = INITIAL_EPSILON

    # outdir = './results'
    # env = gym.wrappers.Monitor(env, directory=outdir, force=True)
    network = DQN(n_actions=N_ACTIONS,
                  img_size=IM_SIZE,
                  gamma=GAMMA,
                  n_state_frames=N_STATE_FRAMES,
                  target_update_cycle=TARGET_UPDATE_CYCLE)
    img_transformer = ImageTransformer(im_size=IM_SIZE,
                                       method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # gaining minimal experience
    network = gain_minimal_experience(model=network,
                                      img_transformer=img_transformer)
    for episode in range(N_EPISODES):
        if epsilon > FINAL_EPSILON:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EPSILON_DECAY

        episode_reward = 0
        obs = env.reset()
        small_obs = img_transformer.transform(obs)
        state = np.stack([small_obs] * 4, axis=2)
        t = 0
        while True:
            # env.render()
            if np.random.uniform() < epsilon:
                action = env.action_space.sample()
            else:
                state = np.expand_dims(state, axis=0)
                tmp = network.evaluation_network(state)
                action = tf.argmax(tmp[0])
            obs, reward, done, _ = env.step(action)
            small_obs = img_transformer.transform(obs)
            state = np.squeeze(state)
            next_state = update_state(state, small_obs)
            network.add_experience(state, action, reward, done, next_state)
            episode_reward += reward

            network.train()
            state = next_state
            if done:
                logging.info(
                    'Episode {} finished after {} timesteps, total rewards {}'.format(episode, t + 1, episode_reward))
                break
            t += 1
    env.close()